In [1]:
import re   

In [2]:
## Read in the Chinese character pinyin map in a dictionary
Chinese_character_pinyin_map = {}
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/charmap',encoding='utf8') as Chinese_charmap:
    for line in Chinese_charmap:
        Chinese_character_pinyin_map[line[0]] = line[1:].strip()
        
count_yi = 0
for char in Chinese_character_pinyin_map:
    if Chinese_character_pinyin_map[char] == 'yi':
        count_yi+=1
        
print ('The number of possible Chinese characters that are pronounced as yi is {}'.format(count_yi))

The number of possible Chinese characters that are pronounced as yi is 484


In [7]:
## Construct a dict, and the key of the dict is the pinyin in Charmap, and the value is a list contains all the Chinese
## characters matching that pinyin
Chinese_pinyin_character_map = {}
for char in Chinese_character_pinyin_map:
    if Chinese_character_pinyin_map[char] not in Chinese_pinyin_character_map:
        Chinese_pinyin_character_map[Chinese_character_pinyin_map[char]] = [char]
    else:
        Chinese_pinyin_character_map[Chinese_character_pinyin_map[char]].append(char)

for pron in Chinese_pinyin_character_map:
    if len(pron) == 1:
        Chinese_pinyin_character_map[pron].append(pron)
        
print (len(Chinese_pinyin_character_map['yi']))
print (Chinese_pinyin_character_map['a'])
#print (Chinese_pinyin_character_map['yi'])

484
['啊', '阿', '嗄', '锕', 'a']


In [13]:
## Read in the Chinese training file as a big string
Chinese_train_string = ''
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/train.han',encoding='utf8') as Chinese_train:
    for line in Chinese_train:
        Chinese_train_string += line
#print (Chinese_train_string[0:100])        

In [14]:
## Construct unigram word count for the train string
word_count_unigram = {} ## This contains the word counts for unigram in Chinese_train file
for cha in Chinese_train_string:
    if cha not in word_count_unigram:
        word_count_unigram[cha]=1
    else:
        word_count_unigram[cha]+=1

In [15]:
### define function to create the tree structure to store the training parameters
def create(ngram, dic):## ngram is a ngram string, dic is a dictionary. 
    n = len(ngram)
    if n == 0: ##base case 
        return
    if ngram[0] not in dic:
        dic[ngram[0]] = []
        dic[ngram[0]].append(1)
        temp_dict = {}
        dic[ngram[0]].append(temp_dict)
    else:
        dic[ngram[0]][0] += 1
    create(ngram[1:],dic[ngram[0]][1])  

In [16]:
## Train the n-gram model
n = 9  ## 10-gram model
ngram_training = {} ## This is tree structure containing the traing results
                    ## Key is the cha, then value is a list; 1st element in the list is the count of the key,
                    ## 2nd element in the list is a dict in which key is the cha left following the previous key
                    ## for n gram, there are n levels of the tree

for i in range(len(Chinese_train_string)-(n-1)):
    ngram = ''
    for j in range(n):
        ngram +=  Chinese_train_string[i+j]
    create(ngram, ngram_training)

#Deal with the last a few characters 
n = n-1
while n!=0 :
    create(Chinese_train_string[-n:],ngram_training)
    n-=1

In [17]:
## define a function to find the counts in the tree structure. This function traverse the tree to find the counts of ngram  
def find_counts (ngram, ngram_training): ## ngram_training is a tree structure for n = 10, so len(ngram) should be less than 10
    n = len(ngram)                       ## find the counts of ngram
    if n == 1:  ## base case
        if ngram not in ngram_training:
            return 0
        else:
            return ngram_training[ngram][0]    
    else:
        if ngram[0] not in ngram_training:
            return 0
        else:
            return find_counts(ngram[1:],ngram_training[ngram[0]][1])

In [18]:
## define a function to find the subtree 
def find_subtree (ngram, ngram_training): ## ngram_training is a tree structure for n = 10, so len(ngram) should be less than 10
    n = len(ngram)
    if n == 1:
        if ngram not in ngram_training:
            temp ={}
            return temp
        else:
            return ngram_training[ngram][1]    
    else:
        if ngram[0] not in ngram_training:
            temp = {}
            return temp
        else:
            return find_subtree(ngram[1:],ngram_training[ngram[0]][1])

In [19]:
## Define the recursive function for prediction with witten-bell smoothing
## This function returns p(w/u)
def prediction_witten_bell_smoothing(n,w,position_predict,word_string_han): ## n means n-gram, w is the character to predict, 
                                                                        ## position_predict is the postion to be predicted                                                                       
    ## construct u according to the correct han characters     
    u=''
    for i in range(n-1,0,-1):
        u += word_string_han[position_predict-i]   
    
        
    if len(u)==0: ## base case, return p(w)  
        N = len(Chinese_train_string) # number of word token-not including <s>
        n_1_plus = len(word_count_unigram)## number of word type seen at least one time -not including <s>
        n_word_type = len(word_count_unigram)+1 ## number of word type_include unkown word
        lamda = N/(N+n_1_plus)
        if w in word_count_unigram:
            return lamda*ngram_training[w][0]/N + (1-lamda)/n_word_type   ## ngram_training[w][0] represents c(w), return p(w)
        else:
            return (1-lamda)/n_word_type
        
    if find_counts (u, ngram_training) == 0:
        return prediction_witten_bell_smoothing(n-1,w,position_predict,word_string_han)
    else:        
        ## calculate c(u_dot) and n1+(u_dot)        
        sub_tree_under_u = find_subtree (u, ngram_training)
        count_u_dot = 0
        for i in sub_tree_under_u:
            count_u_dot+=sub_tree_under_u[i][0]     
        
        count_1plus_u_dot = 0
        for i in sub_tree_under_u:
            count_1plus_u_dot+=1 
            
        if count_u_dot == 0:
            return prediction_witten_bell_smoothing(n-1,w,position_predict,word_string_han)
                
        ## calculate lamda_u
        lamda_u = count_u_dot/(count_u_dot+count_1plus_u_dot)
        
        ## calculate c(uw)
        c_uw = find_counts(u+w, ngram_training)
        
        return lamda_u*c_uw/count_u_dot+(1-lamda_u)*prediction_witten_bell_smoothing(n-1,w,position_predict,word_string_han)

In [20]:
## Read in dev.pin as a big string, and convert to list with all the words 
Chinese_dev_pinyin_string = ""
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/dev.pin',encoding='utf8') as Chinese_dev_pinyin:
    for line in Chinese_dev_pinyin:
        Chinese_dev_pinyin_string += line
        
#print (Chinese_dev_pinyin_string[0:100])
word_dev_pin_list=re.findall(r'\S+|\n',Chinese_dev_pinyin_string)
#'\S+|\n' will match all the combinations of none whitespace character with length 1 or more (\S+) or new line (\n).
#word_dev_pin_list = Chinese_dev_pinyin_string.split()
#print (word_dev_pin_list)

In [21]:
## Read in dev.han as a big string, remove all the newline character 
Chinese_dev_han_string = ""
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/dev.han',encoding='utf8') as Chinese_dev_han:
    for line in Chinese_dev_han:
        Chinese_dev_han_string += line
print (len(Chinese_dev_han_string))

1816


In [22]:
## Make prediction on the dev file
n=9
Chinese_prediction = {}
for position in range(n-1, len(word_dev_pin_list)):
    prediction_word = {}
    if word_dev_pin_list[position] == '<space>':
        Chinese_prediction[position] = " "
    else:
        if word_dev_pin_list[position] in Chinese_pinyin_character_map:
            for word in Chinese_pinyin_character_map[word_dev_pin_list[position]]: 
                prediction_word[word] = prediction_witten_bell_smoothing(n,word,position,Chinese_dev_han_string)
            Chinese_prediction[position]=[key for key, val in prediction_word.items() if val == max(prediction_word.values())][0]
        else: 
            Chinese_prediction[position] = word_dev_pin_list[position]
            
Correct_prediction_count = 0
Correct_position = []
for position in range(n-1, len(word_dev_pin_list)):
    if Chinese_prediction[position] == Chinese_dev_han_string[position]:
        Correct_prediction_count+=1
        Correct_position.append(position)

print (Correct_prediction_count/(len(Chinese_dev_han_string)-(n-1)))


0.9242256637168141


In [23]:
## Read in test.pin as a big string, and convert to list with all the words 
Chinese_test_pinyin_string = ""
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/test.pin',encoding='utf8') as Chinese_test_pinyin:
    for line in Chinese_test_pinyin:
        Chinese_test_pinyin_string += line
        
word_test_pin_list = re.findall(r'\S+|\n',Chinese_test_pinyin_string)
print (len(word_test_pin_list))

1646


In [24]:
## Read in dev.han as a big string, remove all the newline character 
Chinese_test_han_string = ""
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/chinese/test.han',encoding='utf8') as Chinese_test_han:
    for line in Chinese_test_han:
        Chinese_test_han_string += line
        
print (len(Chinese_test_han_string))

1646


In [26]:
## Make prediction on the test file
n=9
#English_letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                   #'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

Chinese_prediction = {}
for position in range(n-1, len(word_test_pin_list)):
    prediction_word = {}
    if word_test_pin_list[position] == '<space>':
        Chinese_prediction[position] = " "
        
    #elif word_test_pin_list[position] in English_letters:
        #Chinese_prediction[position] = word_test_pin_list[position]
                      
    else:        
        
        if word_test_pin_list[position] in Chinese_pinyin_character_map:
            for word in Chinese_pinyin_character_map[word_test_pin_list[position]]: 
                prediction_word[word] = prediction_witten_bell_smoothing(n,word,position,Chinese_test_han_string)
            Chinese_prediction[position]=[key for key, val in prediction_word.items() if val == max(prediction_word.values())][0]
        else:
            Chinese_prediction[position]=word_test_pin_list[position]
            
Correct_prediction_count = 0
for position in range(n-1, len(word_test_pin_list)):
    if Chinese_prediction[position] == Chinese_test_han_string[position]:
        Correct_prediction_count+=1

print (Correct_prediction_count/(len(Chinese_test_han_string)-(n-1)))

0.8821733821733821
